In [14]:
import talib
import numpy as np
import requests
import pandas as pd
import time
import datetime
from ciso8601 import parse_datetime

from ftx_client import FtxClient

### Notes

- Currently, the price is the last traded price. We may want to change it to the average price in the last window

### Api keys and inits

In [5]:
# api_key_test = 'wiYFjNZa8vQZ00HVlvviBZU8KlUk3P9BsOaFNgWQ'
# api_secret_test = 'FV3I-fMloT9dVIO5W1twDUZA385Q06VDjrN2mqeV'
api_key = '4MZCayt-zZKp3QQFOBgPFdwOK5B1zoxJPMeFAGKz'
api_secret = 'cfQxfctS5j7lR0iq-iGsiItt4znrH9P4AkF-IzMH'

# with open('api_keys.txt', 'r') as f:
#     lines = [x.rstrip('\n') for x in f.readlines()]
#     api_key = lines[0]
#     api_secret = lines[1]

curr_pairs = ['BTC', 'USD']
curr_symbol = '{}/{}'.format(curr_pairs[0], curr_pairs[1])
rsi_resolution = 900 # 15 minutes in seconds

logfile = 'ftx_log.log'
with open(logfile, 'w') as f:
    pass

client = FtxClient(api_key=api_key, api_secret=api_secret)

### Utility Functions

In [7]:
def logTrade(oldBalance, newBalance, branch, enterTime, enterPrice, exitPrice, orderid):
    with open(logfile, 'a') as f:
        f.write(f'oldBalance: {oldBalance}, newBalance: {newBalance}, branch: {branch}, enterTime: {enterTime}, enterPrice: {enterPrice}, exitPrice: {exitPrice}, orderid: {orderid}\n')
        
def Stoch(close,high,low, smoothk, smoothd, n):
    lowestlow = pd.Series.rolling(low,window=n,center=False).min()
    highesthigh = pd.Series.rolling(high, window=n, center=False).max()
    K = pd.Series.rolling(100*((close-lowestlow)/(highesthigh-lowestlow)), window=smoothk).mean()
    D = pd.Series.rolling(K, window=smoothd).mean()

    return K, D

def rsi(prices):
    i = 0
    upPrices=[]
    downPrices=[]
    #  Loop to hold up and down price movements
    while i < len(prices):
        if i == 0:
            upPrices.append(0)
            downPrices.append(0)
        else:
            if (prices[i]-prices[i-1])>0:
                upPrices.append(prices[i]-prices[i-1])
                downPrices.append(0)
            else:
                downPrices.append(prices[i]-prices[i-1])
                upPrices.append(0)
        i += 1
    x = 0
    avg_gain = []
    avg_loss = []
    #  Loop to calculate the average gain and loss
    while x < len(upPrices):
        if x <15:
            avg_gain.append(0)
            avg_loss.append(0)
        else:
            sumGain = 0
            sumLoss = 0
            y = x-14
            while y<=x:
                sumGain += upPrices[y]
                sumLoss += downPrices[y]
                y += 1
            avg_gain.append(sumGain/14)
            avg_loss.append(abs(sumLoss/14))
        x += 1
    p = 0
    RS = []
    RSI = []
    #  Loop to calculate RSI and RS
    while p < len(prices):
        if p <15:
            RS.append(0)
            RSI.append(0)
        else:
            RSvalue = (avg_gain[p]/avg_loss[p])
            RS.append(RSvalue)
            RSI.append(100 - (100/(1+RSvalue)))
        p+=1
        
#     print('RSI: ', RSI)
    
    return RSI

def get_RSI_df():
#     candles = client.get_klines(symbol=curr_symbol, interval=Client.KLINE_INTERVAL_15MINUTE)
    candles = client.get_candles(curr_symbol, rsi_resolution)
    opens = [x['open'] for x in candles]
    closes = [x['close'] for x in candles]
    highs = [x['high'] for x in candles]
    lows = [x['low'] for x in candles]
    startTimes = [x['time'] for x in candles]
    volumes = [x['volume'] for x in candles]
    df = pd.DataFrame(list(zip(opens, closes, highs, lows, startTimes, volumes)), columns=['open', 'close', 'high', 'low', 'startTime', 'volume'])
#     df.columns=['timestart','open','high','low','close','?','timeend','?','?','?','?','?']
#     df.timestart = [datetime.datetime.fromtimestamp(i/1000) for i in df.timestart.values]
#     df.timeend = [datetime.datetime.fromtimestamp(i/1000) for i in df.timeend.values]
    
    float_data = [float(x) for x in df.close.values]
    np_float_data = np.array(float_data)
    all_rsi = talib.RSI(np_float_data, 14)
#     float_data.reverse()
#     all_rsi1 = rsi(float_data)
#     print('diff: ', [all_rsi[i] - all_rsi1[i] for i in range(len(all_rsi))])
#     print('all_rsi: ', all_rsi)
#     print('len all_rsi: ', len(all_rsi))
    df['rsi'] = all_rsi
#     print('rsi: ', df['rsi'].astype(float).iloc[-1])

    stochrsi = Stoch(df.rsi, df.rsi, df.rsi, 3, 3, 14)
    df['StochrsiK'],df['StochrsiD'] = stochrsi
    print('stochrsiK: ', df['StochrsiK'][-15:])
    
    return df

def get_RSI():
    df = get_RSI_df()
    newestK = df.StochrsiK.astype(float).iloc[-1]
    return newestK

def print_log(s):
    print(s)
    with open(logfile, 'a') as f:
        f.write(s + '\n')

In [46]:
print(get_RSI())

diff:  [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 37.60806916426513, -39.55700306927741, -44.14810187338922, -45.83575454006078, -54.91433005435205, -50.550165128442885, -42.65841470370893, -36.14536928101076, -33.63871490246884, -26.153855320878172, -27.147384309895706, -35.00549916837801, -32.895340834664395, -32.97085110086283, -18.256380684780204, -20.104285335038746, -24.70096272922254, -23.084379243421495, -20.921629712334493, -17.94824164349309, -14.571172637965034, 5.115251367897692, 4.181365627319195, 3.8120718952712167, 5.897372745729584, 6.383916756274317, 11.694433014887188, 15.901589238134584, 18.435765879620376, 19.98248351358479, 21.398750764667398, 24.755847963604452, 25.901725228053195, 14.95962233426718, 20.105024669260608, 22.460642345127823, 24.591500929651062, 28.053359609214567, 15.298345755904656, -4.12522121926353, -4.864895326561232, 2.8329311356723963, 2.0136786331665704, -10.801346174164777, -18.149095249259453, -15.365776463145103

### Testing (Check balance)

In [9]:
price = client.get_price(curr_symbol)
print(price)
print(client.get_account_info())
print()
print('hi, ', client.get_balance(curr_pairs[1]))
print('bye, ', client.get_balances())
print()
candles = client.get_candles(curr_symbol, rsi_resolution)
print(len(candles))
print(candles[3])
print(parse_datetime(candles[3]['startTime']).timestamp())
print(get_RSI())
print('#############')
print()
# print(client.get_order_status(7204332489))

13515.0
{'backstopProvider': False, 'chargeInterestOnNegativeUsd': False, 'collateral': 2644.115910590172, 'freeCollateral': 2644.106070970914, 'initialMarginRequirement': 0.1, 'leverage': 10.0, 'liquidating': False, 'maintenanceMarginRequirement': 0.03, 'makerFee': 0.0, 'marginFraction': None, 'openMarginFraction': None, 'positionLimit': None, 'positionLimitUsed': None, 'positions': [], 'spotLendingEnabled': False, 'spotMarginEnabled': False, 'takerFee': 0.0007, 'totalAccountValue': 2644.115910590172, 'totalPositionSize': 0.0, 'useFttCollateral': True, 'username': 'quant@sminem.com'}

hi,  2643.73216562
bye,  [{'coin': 'BTC', 'free': 2.913e-05, 'total': 2.913e-05, 'usdValue': 0.3935981518413195}, {'coin': 'USD', 'free': 2643.73216562, 'total': 2643.73216562, 'usdValue': 2643.73216562476}]

1500
{'close': 11378.0, 'high': 11385.5, 'low': 11375.5, 'open': 11381.5, 'startTime': '2020-10-18T02:15:00+00:00', 'time': 1602987300000.0, 'volume': 311316.55695}
1602987300.0
stochrsiK:  1485    

### Testing (Place/cancel order)

In [12]:
# Place order
print(price)
enterLong(price)
# exitLong()
# client.cancel_order(7226227238)

13515.0
res:  {'avgFillPrice': None, 'clientId': None, 'createdAt': '2020-11-02T16:16:12.527842+00:00', 'filledSize': 0.0, 'future': None, 'id': 13184027008, 'ioc': False, 'liquidation': False, 'market': 'BTC/USD', 'postOnly': False, 'price': 13515.0, 'reduceOnly': False, 'remainingSize': 0.1955, 'side': 'buy', 'size': 0.1955, 'status': 'new', 'type': 'limit'}


KeyboardInterrupt: 

### Testing (Check open orders, order history)

In [29]:
# Check order status
def print_side_orders(side):
    orders = client.get_order_history(market=curr_symbol)
    for order in orders:
        if order['side'] == side:
            print(order)

orders = client.get_open_orders(market=curr_symbol)
print('open orders: ', orders)
history = client.get_order_history(market=curr_symbol)
print(len(history))
print(history)
print()
# print(history[0])
# print(history[1])
print()

print_side_orders('buy')
print()
print_side_orders('sell')

# exitLong(price)

open orders:  []
7
[{'avgFillPrice': 10845.0, 'clientId': None, 'createdAt': '2020-09-17T16:10:39.034129+00:00', 'filledSize': 0.2439, 'future': None, 'id': 9899980860, 'ioc': False, 'liquidation': False, 'market': 'BTC/USD', 'postOnly': False, 'price': 10844.5, 'reduceOnly': False, 'remainingSize': 0.0, 'side': 'sell', 'size': 0.2439, 'status': 'closed', 'type': 'limit'}, {'avgFillPrice': 10850.0, 'clientId': None, 'createdAt': '2020-09-17T16:10:33.300360+00:00', 'filledSize': 0.2441, 'future': None, 'id': 9899975728, 'ioc': False, 'liquidation': False, 'market': 'BTC/USD', 'postOnly': False, 'price': 10850.0, 'reduceOnly': False, 'remainingSize': 0.0, 'side': 'buy', 'size': 0.2441, 'status': 'closed', 'type': 'limit'}, {'avgFillPrice': 10846.637740483013, 'clientId': None, 'createdAt': '2020-09-17T15:44:08.379008+00:00', 'filledSize': 0.2443, 'future': None, 'id': 9898692721, 'ioc': False, 'liquidation': False, 'market': 'BTC/USD', 'postOnly': False, 'price': 10845.0, 'reduceOnly': F

### Trading Functions

In [11]:
def enterLong(price):
    curr_usd = client.get_balance('USD')
    usd_per_btc = client.get_price(curr_symbol)
    btc = curr_usd / usd_per_btc
    newprice = price + 1
    
    # Market order
    result = client.place_order(market=curr_symbol, side='buy', price=newprice, size=btc, type='limit')
    print('res: ', result)
    orderid = result['id']
    
    # Wait for market order to get filled
    start_time = time.time()
    while True:
        try:
            status = client.get_order_status(orderid)['status'] 
        except Exception as e:
            print_log('Error in enterLong: ' + repr(e))
            
        if status == 'closed':
            break
            
        if time.time() - start_time > 60 * 60: # 1 hour
            print_log('Waited too long in enterLong')
            temp = 1/0 # Exit the script
            
        time.sleep(3)
        
    return orderid
    
def exitLong():
    curr_btc = client.get_balance('BTC')
#     print('curr_btc: ', curr_btc)
    curr_usdvalue = client.get_usdValue('BTC')
    price = curr_usdvalue / curr_btc
    newprice = price - 5
    
    result = client.place_order(market=curr_symbol, side='sell', price=newprice, size=curr_btc, type='limit', ioc=False)
    print('res: ', result)
    
    orderid = result['id']
    
    # Wait for market order to get filled
#     while client.get_order_status(orderid)['status'] != 'closed':
#         time.sleep(3)
        
    start_time = time.time()
    while True:
        try:
            status = client.get_order_status(orderid)['status'] 
        except Exception as e:
            print_log('Error in enterLong: ' + repr(e))
            
        if status == 'closed':
            break
            
        if time.time() - start_time > 60 * 60: # 1 hour
            print_log('Waited too long in exitLong')
            temp = 1/0
            
        time.sleep(3)
        
    return orderid

### Initialize variables (balance, orderDetails, activeLong, etc)

In [31]:
gap = 100.0
rsi_low_thresh = 33.0

activeLong = False
enterPrice = None
exitPrice = None
enterTime = None

### Main Loop

In [32]:
while True:
    time.sleep(5)
    
    try:
        currPrice = client.get_price(curr_symbol)
    except Exception as e:
        print_log('Error in currPrice: ' + repr(e))
        continue
    nearestMult = (currPrice//gap) * gap
    modPrice = currPrice - nearestMult
    centPrice = currPrice // gap
    
    currTime = int(time.time())
    
    # Get RSI
    rsi = get_RSI()
    print('stochRSI: ', rsi)
#     print(type(newestK))
    
    if not activeLong and rsi < rsi_low_thresh and modPrice <= 0.2*gap:
        oldBalance = client.get_balance('USD')
        orderid = enterLong(currPrice)
        print('Entering long with {} USD at {}'.format(oldBalance, currPrice))
        activeLong = True
        enterPrice = currPrice + 1
        enterTime = int(time.time())
        humanTime = datetime.datetime.fromtimestamp(enterTime).strftime('%Y-%m-%d %H:%M:%S')
        print_log('Entering long with {} USD at {} at {} orderid = {}'.format(oldBalance, enterPrice, humanTime, orderid))
        
    if activeLong and enterPrice < currPrice and modPrice >= 0.8*gap:
        orderid = exitLong()
        print('Exiting long at {}'.format(currPrice))
        humanTime = datetime.datetime.fromtimestamp(currTime).strftime('%Y-%m-%d %H:%M:%S')
        logTrade(oldBalance, newBalance=client.get_balance('USD'), branch='Success', enterTime=humanTime, enterPrice=enterPrice, exitPrice=currPrice, orderid=orderid)
        activeLong = False
        
    if activeLong and enterPrice - currPrice > 0.5*gap:
        orderid = exitLong()
        exitPrice = currPrice - 5
        print('Exiting long at {}'.format(exitPrice))
        activeLong = False
        humanTime = datetime.datetime.fromtimestamp(enterTime).strftime('%Y-%m-%d %H:%M:%S')
        logTrade(oldBalance, newBalance=client.get_balance('USD'), branch='Cut loss', enterTime=humanTime, enterPrice=enterPrice, exitPrice=currPrice, orderid=orderid)

stochRSI:  74.69567575671303
stochRSI:  74.69567575671303
stochRSI:  74.69567575671303
stochRSI:  74.69567575671303
stochRSI:  74.69567575671303


KeyboardInterrupt: 